# Wanderer
> Play Wanderer

Wanderer is sourced from github.com/red-kangaroo/wanderer as a `git subtree`.
We also patch it slightly to make it easier to instrument.

## Interface via Selenium

In [18]:
from selenium.webdriver.common.keys import Keys
import random
import re
import io
import sys

lm = [
    Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT,
    '.', 'x'
]

#################
# Random Strategy
# effectively, causes staff to be used less often
# longer games, but not necessarily more points.
costs = [1/cost for cost in [1,1,1,1,1,10]]
def random_move(b):
    return random.choices(lm, costs)
#################

#################
# Manual Strategy [WIP]
def human_move(b):
    return sys.stdin.read(1)
#################

def board(g):
    # .text is a little too magical (writes \n instead of <br>, etc..)
    return g.find_element(by="id", value="tsv").get_attribute('innerHTML')

def game_score(g):
    # .text is easier to regex than innerHTML
    game_over = re.search("Your final score is (\d+) after ", g.text)
    # scoring function is the score, but alternative scoring
    # functions could count kills, visits, keys, etc..
    return game_over.group(1) if game_over else None

def turn(g):
    s = game_score(g)
    b = board(g)
    # How we get m will by the strategy
    m = random_move(b)
    # m = human_move(board(g))
    if not s: g.send_keys(m)
    return (s, f"{b}\t{m}")

def turn_score(b, g):
    try:
        return int(re.match("Self: (?:\<[^>]+\>)*(\w+)", b.split("\t")[2500]).group(1))
    except ValueError:
        return int(re.search("Your final score is (\d+) after ", g.text).group(1))

def reward(b1, b2, g):
    return turn_score(b2, g) - turn_score(b1, g)

def game(driver, save_as=None):
    driver.get("file:///Users/ewolfson/dev/github.com/yegeniy/rougelaike/wanderer/src/Wanderer.html")
    g = driver.find_element(by="tag name", value="body")
    score = None
    history = []
    while score is None:
        score, b = turn(g)
        r = reward(b, board(g), g)
        # make sure not to leak the moves and rewards in training
        history.append(f"{b}\t{r}")
    # print(history)
    if save_as:
        with open(f"{save_as}", "w") as f: f.write("\n".join(history))
    return int(score), len(history)

Initialize the Selenium driver.
Unless you use headless mode, you'll want to keep this new Firefox window open as long as you are playing.

In [2]:
from selenium import webdriver
#driver = webdriver.Firefox()
#try: driver.quit()
#except: pass
from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [19]:
! mkdir /tmp/wanderer
%time game(driver, "/tmp/wanderer/game")

mkdir: /tmp/wanderer: File exists
CPU times: user 422 ms, sys: 22.3 ms, total: 444 ms
Wall time: 6.47 s


(0, 74)

In [192]:
from time import time

def play_past(min_score, save_dir=None):
    ts=int(time())
    g=0
    while(True and g < 100):
        score, turns = game(driver, f"{save_dir}/{ts}_{g}.tsv" if save_dir else None)
        g+=1
        if score > min_score:
            print(f"{save_dir}/{ts}, {score=}, {turns=} after {g} games")
            break

In [193]:
%time play_past(0, "/tmp/wanderer/")

/tmp/wanderer//1597558645, score=5, turns=82 after 14 games
CPU times: user 6.94 s, sys: 389 ms, total: 7.33 s
Wall time: 2min 30s


It might not always be possible to record enough games to show how to get more than a score of 5 using a random strategy. Basically, the cell below will run for a very long time. Possibly long enough to run into a bug in the game.

In [153]:
play_past(27)

- Could consider recording manual runs to bootstrap the test set.
- Need to figure out how to record game state.. Maybe just write a line of `g.text` down or its diff?

### Debugging Selenium


In [ ]:
b.get_current_page().getText()

In [5]:
'\n\nWanderer\n\n\n\n\n\n\n\n\n\r\n    Wanderer v2.1\r\n  \n\r\n    There are so many worlds to explore, but only so much of you left...\r\n  \r\n    Welcome, wanderer. You can check out the source code and report issues on\r\n    GitHub, or post\r\n    a comment on my blog.\r\n    Have fun!\r\n  \r\n    Use F11 for fullscreen mode.\r\n  \r\n    Use Ctrl + and Ctrl - to adjust the size\r\n    to your liking.\r\n  \r\n    Press ? for game help.\r\n  \n\n\n\n\n\n'

'\n\nWanderer\n\n\n\n\n\n\n\n\n\r\n    Wanderer v2.1\r\n  \n\r\n    There are so many worlds to explore, but only so much of you left...\r\n  \r\n    Welcome, wanderer. You can check out the source code and report issues on\r\n    GitHub, or post\r\n    a comment on my blog.\r\n    Have fun!\r\n  \r\n    Use F11 for fullscreen mode.\r\n  \r\n    Use Ctrl + and Ctrl - to adjust the size\r\n    to your liking.\r\n  \r\n    Press ? for game help.\r\n  \n\n\n\n\n\n'

In [210]:
random.choices(lm, costs, k=100)

['\ue015',
 '\ue015',
 '\ue013',
 '.',
 '.',
 '\ue013',
 '.',
 '\ue013',
 '\ue012',
 '\ue014',
 '\ue015',
 '.',
 '\ue013',
 '\ue012',
 '\ue013',
 '\ue014',
 '.',
 '.',
 '.',
 '\ue013',
 '\ue014',
 'x',
 '\ue013',
 '\ue012',
 '\ue015',
 '\ue014',
 '\ue013',
 '\ue012',
 '\ue014',
 '\ue015',
 '\ue013',
 '\ue015',
 '\ue013',
 '\ue013',
 '\ue014',
 '\ue015',
 '\ue013',
 '\ue013',
 '\ue012',
 '\ue012',
 '\ue014',
 '.',
 '\ue015',
 '\ue013',
 'x',
 '\ue012',
 '.',
 '.',
 '\ue012',
 '\ue013',
 '\ue015',
 '\ue012',
 '\ue012',
 '\ue013',
 '\ue014',
 '.',
 '.',
 '\ue014',
 '\ue014',
 '\ue012',
 '\ue012',
 '\ue013',
 '\ue012',
 '\ue012',
 '\ue015',
 '\ue014',
 '\ue012',
 '\ue013',
 '\ue012',
 '\ue012',
 '\ue012',
 '\ue014',
 '\ue012',
 '\ue013',
 '\ue013',
 '\ue012',
 '\ue012',
 '.',
 '\ue014',
 '\ue014',
 '\ue013',
 '\ue015',
 '\ue014',
 '\ue013',
 '\ue013',
 '\ue013',
 '\ue015',
 '\ue015',
 '\ue013',
 '\ue015',
 '\ue013',
 '\ue014',
 '\ue013',
 '\ue014',
 '\ue013',
 '\ue014',
 '\ue014',
 '\ue015

In [209]:
'x' in random.choices(lm, costs, k=100)

True

In [139]:
! ls -lathr /tmp/wanderer | tail

-rw-r--r--   1 ewolfson  wheel   682K Aug 15 20:26 1.tsv
-rw-r--r--   1 ewolfson  wheel   486K Aug 15 20:26 2.tsv
-rw-r--r--   1 ewolfson  wheel   610K Aug 15 20:26 3.tsv
-rw-r--r--   1 ewolfson  wheel   878K Aug 15 20:27 4.tsv
-rw-r--r--   1 ewolfson  wheel   496K Aug 15 20:27 5.tsv
-rw-r--r--@  1 ewolfson  wheel   422K Aug 15 20:31 6.tsv
drwxrwxrwt   6 root      wheel   192B Aug 16 00:25 ..
-rw-r--r--   1 ewolfson  wheel   1.6M Aug 16 00:41 game.tsv
drwxr-xr-x  11 ewolfson  wheel   352B Aug 16 00:44 .
-rw-r--r--   1 ewolfson  wheel   1.6M Aug 16 00:44 game.tsv.tar


In [138]:
! tar -cf /tmp/wanderer/game.tsv.tar /tmp/wanderer/game.tsv

tar: Removing leading '/' from member names


In [134]:
! tail -n 1 /tmp/wanderer/game.tsv 

█	█	█	&nbsp;	█	█	█	█	█	█	█	█	&nbsp;	█	█	█	█	█	█	█	█	█	█	█	█	█	&nbsp;	&nbsp;	█	&nbsp;	█	&nbsp;	█	█	█	&nbsp;	&nbsp;	█	&nbsp;	█	█	█	█	█	█	&nbsp;	█	&nbsp;	█	█	█	█	█	█	&nbsp;	█	█	&nbsp;	█	█	&nbsp;	█	█	█	█	&nbsp;	█	&nbsp;	&nbsp;	█	█	&nbsp;	█	█	█	&nbsp;	█	█	&nbsp;	█	█	█	█	█	█	█	█	&nbsp;	█	█	█	█	█	&nbsp;	█	█	█	█	█	█	█	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	█	█	&nbsp;	█	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	█	█	█	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	█	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	█	&nbsp;	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	█	█	█	&nbsp;	&nbsp;	█	&nbsp;	█	█	&nbsp;	,	█	&nbsp;	█	&nbsp;	&nbsp;	█	█	&nbsp;	&nbsp;	█	█	&nbsp;	&nbsp;	█	&nbsp;	█	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;	&nbsp;	&nbsp;	█	█	█	&nbsp;	&nbsp;	&nbsp;	█	█	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	<t style="background-color:#1E90FF">~</t>	&nbsp;	&nbsp;	█	█	█	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	&nbsp;	█	&nbsp;	&nbsp;

# EOF